# Advent of Code 2024

## --- Day 1: Historian Hysteria ---

In [1]:
with open('day_1_input.txt') as input:
    lines = [line.rstrip() for line in input.readlines()]

line_pairs = [tuple(int(num) for num in line.split()) for line in lines]
list1, list2 = [sorted(list(x)) for x in zip(*line_pairs)]
# print(list1, list2)

diffs = [abs(val1 - val2) for val1, val2 in zip(list1, list2)]
# print(diffs)

print("Part 1 Result:", sum(diffs))

Part 1 Result: 2904518


In [2]:
scores = [(val1 * sum([val2 == val1 for val2 in list2])) for val1 in list1]
# print(scores)

print("Part 2 Result:", sum(scores))

Part 2 Result: 18650129


## --- Day 2: Red-Nosed Reports ---

In [3]:
with open('day_2_input.txt') as input:
    lines = [line.rstrip() for line in input.readlines()]

reports = [[int(num) for num in line.split()] for line in lines]
# print(reports)

diffs = [[y - x for x, y in zip(report[:-1], report[1:])] for report in reports]
# print(diffs)

steps = [[min(abs(val), 4) for val in diff] for diff in diffs]
# print(steps)

unsafe_steps = [4 in step or 0 in step for step in steps]
# print(unsafe_steps)

unsafe_changes = [abs(sum([int(val > 0) - int(val < 0) for val in diff])) != len(diff) for diff in diffs]
# print(unsafe_changes)

safe = [not unsafe_step and not unsafe_change for unsafe_step, unsafe_change in zip(unsafe_steps, unsafe_changes)]
# print(safe)

print("Part 1 Result:", sum(safe))

Part 1 Result: 334


In [4]:
def check_for_bad_report(report):
    diff = [y - x for x, y in zip(report[:-1], report[1:])]
    return any(abs(val) > 3 or val == 0 or val * diff[0] < 0 for val in diff)

tally = 0
for report in reports:
    if not check_for_bad_report(report):
        tally += 1
        continue

    for n in range(len(report)):
        new_report = [val for pos, val in enumerate(report) if pos != n]
        if not check_for_bad_report(new_report):
            tally += 1
            break
    
print("Part 2 Result:", tally)

Part 2 Result: 400


## --- Day 3: Mull It Over ---

In [5]:
with open('day_3_input.txt') as input:
    data = input.read()

def mul_and_sum(text):
    candidates = [segment.split(')')[0].split(',') for segment in text.split('mul(')]
    candidates = [candidate for candidate in candidates if len(candidate) == 2]
    products = [int(candidate[0]) * int(candidate[1]) for candidate in candidates if candidate[0].isdigit() and candidate[1].isdigit()]
    return sum(products)

print("Part 1 Result:", mul_and_sum(data))

Part 1 Result: 167650499


In [6]:
segments = data.split("don't()")
result = mul_and_sum(segments[0])
for segment in segments[1:]:
    for subsegment in segment.split('do()')[1:]:
        result += mul_and_sum(subsegment)

print("Part 2 Result:", result)

Part 2 Result: 95846796


## --- Day 4: Ceres Search ---

In [7]:
with open('day_4_input.txt') as input:
    lines = [line.rstrip() for line in input.readlines()]

word = 'XMAS'

def print_grid(some_grid):
    for row in some_grid:
        print(''.join(row))

grid = [[c for c in line] for line in lines]
# print_grid(grid)

start_pos = []
for row_num, row in enumerate(grid):
    for col_num, c in enumerate(row):
        if c == word[0]:
            start_pos.append((row_num, col_num))

dir_list = [(-1, -1), (0, -1), (1, -1), (-1, 0), (1, 0), (-1, 1), (0, 1), (1, 1)]

def sum_pos(a, b):
    return (a[0] + b[0], a[1] + b[1])
def valid_pos(some_grid, p):
    return p[0] >= 0 and p[0] < len(some_grid) and p[1] >= 0 and p[1] < len(some_grid[0])
def grid_get(some_grid, p):
    return some_grid[p[0]][p[1]]

current_pos = []
for pos in start_pos:
    for dir in dir_list:
        check_pos = sum_pos(pos, dir)
        if valid_pos(grid, check_pos) and grid_get(grid, check_pos) == word[1]:
            current_pos.append((check_pos, dir))

next_pos = []
for pos, dir in current_pos:
    check_pos = sum_pos(pos, dir)
    if valid_pos(grid, check_pos) and grid_get(grid, check_pos) == word[2]:
        next_pos.append((check_pos, dir))

current_pos = next_pos
next_pos = []
for pos, dir in current_pos:
    check_pos = sum_pos(pos, dir)
    if valid_pos(grid, check_pos) and grid_get(grid, check_pos) == word[3]:
        next_pos.append((check_pos, dir))

print("Part 1 Result:", len(next_pos))

Part 1 Result: 2644


In [8]:
# print_grid(grid)

word = 'MAS'

start_pos = []
for row_num in range(1, len(grid) - 1):
    row = grid[row_num]
    for col_num in range(1, len(row) - 1):
        if row[col_num] == word[1]:
            start_pos.append((row_num, col_num))

dir_list = [(-1, -1), (1, -1), (-1, 1), (1, 1)]

def negate_pos(p):
    return (-p[0], -p[1])

tally = 0
for pos in start_pos:
    matches = []
    for dir in dir_list:
        check_pos = sum_pos(pos, dir)
        if valid_pos(grid, check_pos) and grid_get(grid, check_pos) == word[0]:
            matches.append(dir)
    if len(matches) != 2 or matches[0] == negate_pos(matches[1]):
        continue
    num_matched = 0
    for dir in matches:
        check_pos = sum_pos(pos, negate_pos(dir))
        if valid_pos(grid, check_pos) and grid_get(grid, check_pos) == word[2]:
            num_matched += 1
    if num_matched == 2:
        # print(pos)
        tally += 1
 
print("Part 2 Result:", tally)

Part 2 Result: 1952


## --- Day 5: Print Queue ---

In [9]:
with open('day_5_input.txt') as input:
    lines = [line.rstrip() for line in input.readlines()]

rules = []
updates = []

for line in lines:
    if '|' in line:
        a, b = line.split('|')
        rules.append((int(a), int(b)))
    elif ',' in line:
        updates.append(tuple(int(val) for val in line.split(',')))

tally = 0
wrong_updates = []
for update in updates:
    page_pos = {page:pos for pos, page in enumerate(update)}
    incorrect = any(earlier in page_pos and later in page_pos and page_pos[earlier] > page_pos[later] for earlier, later in rules)
    if not incorrect:
        tally += update[len(update) // 2]
    else:
        wrong_updates.append(update)

print("Part 1 Result:", tally)

Part 1 Result: 4905


In [10]:
class Page:
    def __init__(self, val):
        self.val = val
    def __lt__(self, other):
        return (self.val, other.val) in rules
    def __repr__(self):
        return f"{self.val}"
        
tally = 0
for update in wrong_updates:
    page_update = sorted(Page(val) for val in update)
    tally += page_update[len(page_update) // 2].val

print("Part 2 Result:", tally)

Part 2 Result: 6204


## --- Day 6: Guard Gallivant ---

In [11]:
with open('day_6_input.txt') as input:
    lines = [line.rstrip() for line in input.readlines()]

def print_grid(some_grid):
    for row in some_grid:
        print(''.join(row))

def add_pos(a, b):
    return (a[0] + b[0], a[1] + b[1])
def sub_pos(a, b):
    return (a[0] - b[0], a[1] - b[1])
def valid_pos(some_grid, p):
    return p[1] >= 0 and p[1] < len(some_grid) and p[0] >= 0 and p[0] < len(some_grid[p[1]])
def grid_get(some_grid, p):
    return some_grid[p[1]][p[0]]
def grid_set(some_grid, p, c):
    some_grid[p[1]][p[0]] = c

grid = [[c for c in line] for line in lines]
# print_grid(grid)

dir_sequence = {(0, -1): 0, (1, 0): 1, (0, 1): 2, (-1, 0): 3}
dir_lookup = {val: key for key, val in dir_sequence.items()}

def next_dir(dir):
    return dir_lookup[(dir_sequence[dir] + 1) % 4]

dir = dir_lookup[0]
start_pos = None
for y in range(len(grid)):
    for x in range(len(grid[0])):
        if grid_get(grid, (x, y)) == '^':
            start_pos = (x, y)
            break
    if not start_pos is None:
        break

progress_grid = [row.copy() for row in grid]
grid_set(progress_grid, start_pos, 'X')

pos = add_pos(start_pos, dir)
while valid_pos(grid, pos):
    if grid_get(grid, pos) == '#':
        pos = sub_pos(pos, dir)
        dir = next_dir(dir)
    else:
        grid_set(progress_grid, pos, 'X')
    pos = add_pos(pos, dir)

# print()
# print_grid(progress_grid)

tally = sum(sum(c == 'X' for c in row) for row in progress_grid)

print("Part 1 Result:", tally)

Part 1 Result: 4647


In [12]:
def check_loop(some_grid, pos, dir):
    history = {(pos, dir)}
    obstruction = add_pos(pos, dir)
    next_pos = obstruction
    while valid_pos(some_grid, next_pos):
        if (next_pos, dir) in history:
            return True
        if next_pos == obstruction or grid_get(some_grid, next_pos) == '#':
            dir = next_dir(dir)
        else:
            pos = next_pos
        history.add((pos, dir))
        next_pos = add_pos(pos, dir)
    return False

progress_grid = [row.copy() for row in grid]
grid_set(progress_grid, start_pos, 'X')

options = set()
dir = dir_lookup[0]
pos = start_pos
next_pos = add_pos(pos, dir)
while valid_pos(grid, next_pos):
    if grid_get(grid, next_pos) == '#':
        dir = next_dir(dir)
    else:
        if next_pos != start_pos and grid_get(progress_grid, next_pos) != 'X' and check_loop(grid, pos, dir):
            options.add(next_pos)
        pos = next_pos
        grid_set(progress_grid, pos, 'X')
    next_pos = add_pos(pos, dir)

# print_grid(progress_grid)
# print(options)

print("tally =", sum(sum(c == 'X' for c in row) for row in progress_grid))
print("Part 2 Result:", len(options))

tally = 4647
Part 2 Result: 1723


## --- Day 7: Bridge Repair ---

In [13]:
with open('day_7_input.txt') as input:
    lines = [line.rstrip() for line in input.readlines()]

lines = [(int(line.split(':')[0]), line.split(': ')[1]) for line in lines]
lines = [(val, [int(arg) for arg in args.split(' ')]) for val, args in lines]

tally = 0
for val, args in lines:
    results = {args[0]}
    for arg in args[1:]:
        new_results = set()
        for result in results:
            new_results.add(result + arg)
            new_results.add(result * arg)
        results = new_results
    if val in results:
        tally += val

print("Part 1 Result:", tally)

Part 1 Result: 7885693428401


In [14]:
tally = 0
for val, args in lines:
    results = {args[0]}
    for arg in args[1:]:
        new_results = set()
        for result in results:
            new_results.add(result + arg)
            new_results.add(result * arg)
            new_results.add(int(str(result) + str(arg)))
        results = new_results
    if val in results:
        tally += val

print("Part 2 Result:", tally)

Part 2 Result: 348360680516005


## --- Day 8: Resonant Collinearity ---

In [15]:
with open('day_8_input.txt') as input:
    grid = [[c for c in line.rstrip()] for line in input.readlines()]

def print_grid(some_grid):
    for row in some_grid:
        print(''.join(row))
# print_grid(grid)

antennas = {}
for y, row in enumerate(grid):
    for x, c in enumerate(row):
        pos = x, y
        if c != '.':
            if c in antennas:
                antennas[c].append(pos)
            else:
                antennas[c] = [pos]

def add_pos(a, b):
    return a[0] + b[0], a[1] + b[1]
def sub_pos(a, b):
    return a[0] - b[0], a[1] - b[1]
def valid_pos(some_grid, pos):
    return pos[0] >= 0 and pos[1] >= 0 and pos[1] < len(some_grid) and pos[0] < len(some_grid[pos[1]])    
def grid_set(some_grid, pos, c):
    some_grid[pos[1]][pos[0]] = c

nodes = set()
for _, antenna_pos in antennas.items():
    for m, m_pos in enumerate(antenna_pos):
        for n in range(m):
            n_pos = antenna_pos[n]
            delta = sub_pos(m_pos, n_pos)
            node1 = add_pos(m_pos, delta)
            node2 = sub_pos(n_pos, delta)
            if valid_pos(grid, node1):
                # grid_set(grid, node1, '#')
                nodes.add(node1)
            if valid_pos(grid, node2):
                # grid_set(grid, node2, '#')
                nodes.add(node2)

# print_grid(grid)
print("Part 1 Result:", len(nodes))

Part 1 Result: 265


In [16]:
nodes = set()
for _, antenna_pos in antennas.items():
    for m, m_pos in enumerate(antenna_pos):
        for n in range(m):
            n_pos = antenna_pos[n]
            delta = sub_pos(m_pos, n_pos)
            node = sub_pos(m_pos, delta)
            while valid_pos(grid, node):
                # grid_set(grid, node, '#')
                nodes.add(node)
                node = sub_pos(node, delta)
            node = add_pos(n_pos, delta)
            while valid_pos(grid, node):
                # grid_set(grid, node, '#')
                nodes.add(node)
                node = add_pos(node, delta)

# print_grid(grid)
print("Part 2 Result:", len(nodes))

Part 2 Result: 962


## --- Day 9: Disk Fragmenter ---

In [17]:
with open('day_9_input.txt') as input:
    line = input.read().rstrip()

disk_map = [int(c) for c in line]
# print(disk_map)

def print_disk(some_disk):
    print(''.join(some_disk))

def build_disk(some_disk_map):
    new_disk = ['.' for _ in range(sum(some_disk_map))]
    id = 0
    head = 0
    free_space = False
    for val in disk_map:
        if not free_space:
            block = str(id)
            for n in range(val):
                new_disk[head + n] = block
            id += 1
        head += val
        free_space = not free_space

    return new_disk

disk = build_disk(disk_map)
# print_disk(disk)

def next_space(some_disk, pos):
    while pos < len(some_disk) and some_disk[pos] != '.':
        pos += 1
    return pos if pos < len(some_disk) else None

def prev_occupied(some_disk, pos):
    while pos >= 0 and some_disk[pos] == '.':
        pos -= 1
    return pos if pos >= 0 else None

head = next_space(disk, 0)
back = prev_occupied(disk, len(disk) - 1)

while head and back and head < back:
    disk[head] = disk[back]
    disk[back] = '.'
    head = next_space(disk, head)
    back = prev_occupied(disk, back)

# print_disk(disk)

checksum = sum(int(id) * pos for pos, id in enumerate(disk[:head]))
print("Part 1 Result:", checksum)

Part 1 Result: 6225730762521


In [18]:
disk = build_disk(disk_map)
# print_disk(disk)

file_sizes = disk_map[0::2]
gap_sizes = disk_map[1::2]

files = dict()
gaps = dict()

head = 0
id = 0
free_space = False
for n in range(len(disk_map)):
    val = disk_map[n]
    if not free_space:
        files[id] = head, val
        id += 1
    elif val != 0:
        if not val in gaps:
            gaps[val] = []
        gaps[val].append(head)
    head += val
    free_space = not free_space

# print(files)
# print(gaps)

for id in range(len(files) - 1, -1, -1):
    pos, file_size = files[id]
    candidate_gaps = [(min(gaps[gap]), gap) for gap in gaps if gap >= file_size]
    if not candidate_gaps:
        continue

    gap_pos, gap_size = min(candidate_gaps)
    if gap_pos >= pos:
        continue

    # remove gap that's been filled
    gap_pos_list = gaps[gap_size]
    gap_pos_list.remove(gap_pos)
    if gap_pos_list:
        gaps[gap_size] = gap_pos_list
    else:
        del gaps[gap_size]

    # update gaps dictionary with any new residual gap
    if file_size < gap_size:
        new_gap_size = gap_size - file_size
        new_gap_pos = gap_pos + file_size
        if not new_gap_size in gaps:
            gaps[new_gap_size] = []
        gaps[new_gap_size].append(new_gap_pos)

    # update files dictionary with the change; helps debugging although technically not necessary
    files[id] = gap_pos, file_size
    
    # rewrite disk to reflect change
    block = str(id)
    for n in range(file_size):
        disk[gap_pos + n] = block
        disk[pos + n] = '.'
    # print_disk(disk)

# print(files)
# print(gaps)
# print_disk(disk)

checksum = sum(int(id) * pos for pos, id in enumerate(disk) if id != '.')
print("Part 2 Result:", checksum)

Part 2 Result: 6250605700557


## --- Day 10: Hoof It ---

In [19]:
with open('day_10_input.txt') as input:
    grid = [[c for c in line.rstrip()] for line in input.readlines()]

def print_grid(some_grid):
    for row in some_grid:
        print(''.join(row))

def grid_get(some_grid, pos):
    return some_grid[pos[1]][pos[0]]

def valid_pos(some_grid, pos):
    return pos[1] >= 0 and pos[1] < len(some_grid) and pos[0] >= 0 and pos[0] < len(some_grid[pos[1]])

def add_pos(a, b):
    return (a[0] + b[0], a[1] + b[1])

# print_grid(grid)

trailheads = []
for row in range(len(grid)):
    for col in range(len(grid[row])):
        pos = col, row
        if grid_get(grid, pos) == '0':
            trailheads.append(pos)

dir_list = [(-1, 0), (1, 0), (0, -1), (0, 1)]

trail_positions = [{trailhead} for trailhead in trailheads]

for height in range(9):
    for index, positions in enumerate(trail_positions):
        new_positions = set()
        for pos in positions:
            for dir in dir_list:
                new_pos = add_pos(pos, dir)
                if not valid_pos(grid, new_pos):
                    continue
                if int(grid_get(grid, new_pos)) != height + 1:
                    continue
                new_positions.add(new_pos)
        trail_positions[index] = new_positions

scores = [len(positions) for positions in trail_positions]
# print(scores)

print("Part 1 Result:", sum(scores))

Part 1 Result: 822


In [20]:
trail_positions = [[trailhead] for trailhead in trailheads]

for height in range(9):
    for index, positions in enumerate(trail_positions):
        new_positions = []
        for pos in positions:
            for dir in dir_list:
                new_pos = add_pos(pos, dir)
                if not valid_pos(grid, new_pos):
                    continue
                if int(grid_get(grid, new_pos)) != height + 1:
                    continue
                new_positions.append(new_pos)
        trail_positions[index] = new_positions

scores = [len(positions) for positions in trail_positions]
# print(scores)

print("Part 2 Result:", sum(scores))

Part 2 Result: 1801
